<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/medical-concept-normalization/fine-tuning-experiments/askpatient/Cadec_twadr_00.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

In [2]:
base_dir = 'medical-concept-normalization'

In [3]:
os.mkdir(base_dir)

In [4]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: course-v3.fast.ai


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [7]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [8]:
path = Path('medical-concept-normalization/data_collection')

In [10]:
# training data set

train_csv = path/'TwADR-L.fold-0.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [11]:
# validating data set

valid = pd.read_csv(path/"TwADR-L.fold-0.validation.csv",header=None)
valid.head()

,0,1
0,13,insomnia
1,57,hypertrophic scarring
2,84,paraplegic
3,93,shooting
4,97,screwing with my short term memory


In [12]:
# testing data set

test = pd.read_csv(path/"TwADR-L.fold-0.test.csv",header=None)
test.head()

,0,1
0,13,insomniac
1,60,cigarette withdrawal
2,108,cramps
3,109,make me drunker
4,133,nightmares


In [13]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'TwADR-cadec.csv')

In [14]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [15]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [16]:
data_clas.add_test(test,label=0)


In [17]:
data_lm.save()
data_clas.save()

In [18]:
data_lm.train_ds.vocab.itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 '.',
 ',',
 'i',
 'the',
 'and',
 'to',
 'my',
 'pain',
 'of',
 'in',
 'a',
 'it',
 'me',
 'mood',
 'increased',
 'for',
 'have',
 'was',
 '-',
 'drug',
 'this',
 ')',
 '(',
 'lipitor',
 'on',
 'disorder',
 'not',
 'withdrawal',
 'abnormal',
 'symptoms',
 'anxiety',
 'that',
 'had',
 'is',
 'muscle',
 'weight',
 'with',
 'like',
 'but',
 'decreased',
 "n't",
 'taking',
 'feeling',
 'disorders',
 'up',
 'sleep',
 'behavior',
 'after',
 'syndrome',
 'sleeplessness',
 'so',
 'blood',
 'got',
 'insomnia',
 'all',
 'diseases',
 'am',
 'feel',
 'side',
 'from',
 '!',
 'disturbance',
 'somnolence',
 'depression',
 'finding',
 'back',
 'effects',
 'loss',
 'energy',
 'cholesterol',
 'been',
 'day',
 'heart',
 'body',
 'be',
 'mental',
 'gain',
 'as',
 'skin',
 'nausea',
 'acute',
 '/',
 'take',
 'appetite',
 'about',
 'no',
 'at',
 'do',
 'fatigue',
 'dizziness',
 'off',
 'now',
 'has',
 'mg',
 'irritabl

In [19]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [20]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [21]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.275078,3.455983,0.431948,08:58


In [22]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.260770,3.020249,0.489090,08:53


In [23]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.081868,2.988015,0.493862,08:54


In [24]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.035752,2.959770,0.496150,08:52


In [25]:
learn.save_encoder('TwADR-cadec_fold0_first')

In [26]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('TwADR-cadec_fold0_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,7.233232,6.554132,0.060870,00:24


In [27]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,6.279817,5.492826,0.130435,00:25


In [28]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.755794,4.802615,0.147826,01:07


In [29]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.461421,4.459805,0.182609,01:08
1,5.251167,4.308764,0.200000,01:03


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.954540,4.065209,0.252174,01:08
1,4.777846,4.038539,0.252174,01:01


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.550385,3.950456,0.260870,01:05
1,4.394523,3.851768,0.278261,01:04


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.195397,3.722086,0.278261,01:05
1,4.060951,3.638329,0.278261,01:08


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.879625,3.549569,0.313043,01:04
1,3.735331,3.524523,0.339130,01:01


In [34]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.567021,3.525229,0.304348,01:08
1,3.472496,3.500514,0.321739,01:08
2,3.272350,3.453426,0.321739,01:06
3,3.101677,3.409678,0.330435,01:02


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.935484,3.458913,0.339130,01:03
1,2.886775,3.431721,0.313043,01:04
2,2.712967,3.365712,0.373913,01:08
3,2.556620,3.380217,0.347826,01:11


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.372758,3.339276,0.356522,01:07
1,2.343452,3.310621,0.356522,01:09
2,2.217196,3.352814,0.356522,01:06
3,2.071698,3.433913,0.339130,01:02


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.975398,3.355983,0.391304,01:03
1,1.900709,3.418064,0.347826,01:03
2,1.787240,3.380969,0.356522,01:05
3,1.643925,3.345597,0.356522,01:02


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.554562,3.348886,0.382609,01:11
1,1.549012,3.424731,0.408696,01:03
2,1.443931,3.369582,0.373913,01:10
3,1.314771,3.364643,0.365217,01:04


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.236918,3.395929,0.382609,01:04
1,1.234299,3.572369,0.356522,01:04
2,1.148754,3.438464,0.330435,01:06
3,1.026012,3.385891,0.347826,01:08


In [40]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.936731,3.391649,0.365217,01:08
1,0.988532,3.723442,0.347826,01:10
2,0.948157,3.691957,0.321739,01:05
3,0.854773,3.698786,0.339130,01:04


In [41]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.772723,3.699604,0.365217,01:09
1,0.748338,3.684272,0.356522,01:08


In [42]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.739160,3.776825,0.347826,01:02
1,0.713579,3.738614,0.339130,01:04


In [43]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.678513,3.797195,0.365217,01:03


In [44]:
# save the best model

learn.save_encoder('TwADR-cadec_fold0')

In [45]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.622294,3.834475,0.330435,01:03


In [46]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.596365,3.770068,0.347826,01:08


In [47]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.588880,3.805523,0.373913,01:02


# Part three: Predict on the test dataset

In [48]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [49]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

143
143
[393, 1597, 265, 1578, 1184, 852, 1096, 1295, 231, 231, 233, 233, 233, 233, 249, 249, 1724, 1787, 1925, 1519, 1529, 1099, 644, 1711, 749, 749, 1942, 792, 792, 792, 813, 852, 852, 749, 1356, 1767, 974, 978, 978, 978, 978, 978, 1711, 824, 1007, 2149, 1669, 1767, 1089, 1099, 426, 974, 1136, 1545, 908, 2097, 1099, 1767, 1356, 1942, 1265, 1930, 1295, 1669, 518, 1308, 1340, 1265, 1340, 1265, 1340, 1340, 1356, 1484, 1508, 1701, 1711, 2118, 1597, 1597, 1597, 1597, 1597, 1599, 1096, 1942, 1610, 1099, 1669, 1787, 1531, 1915, 1711, 1718, 1718, 1767, 1727, 1768, 1073, 2149, 1727, 2191, 1767, 1718, 1787, 1787, 2032, 1787, 1787, 1787, 1099, 1798, 1815, 1815, 1727, 1940, 1877, 1884, 825, 1798, 1930, 1936, 1936, 815, 1942, 1942, 2002, 2109, 729, 2018, 1787, 452, 982, 1877, 825, 1787, 2032, 2147, 974, 1727, 1787, 2191, 2191]
[13, 60, 108, 109, 133, 154, 187, 189, 231, 231, 233, 233, 233, 233, 249, 249, 265, 393, 418, 520, 561, 561, 644, 749, 749, 749, 772, 792, 792, 792, 813, 852, 852, 887, 908

In [50]:
correct_num = 0

for i in range(0, len(standard_list)):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

68
0.4755244755244755
